In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import pandas as pd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Área (m²) de Áreas Verdes Públicas por Distrito

In [3]:
data_path = get_data_diretorio()

# Dependências

Este notebook é dependente dos assets resultante dos notebooks 'areas_verdes_publicas' e '../../arborizacao_viaria/carregar_dados/malha_distritos'

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

In [5]:
gdf_areas_verdes = gpd.read_parquet(
    os.path.join(
        assets_path,
        'areas_verdes_publicas',
        'areas_verdes_publicas.parquet'
    )
)

In [6]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'distrito_ibge.parquet'
    )
)

# Recortar Áreas Verdes por Distrito

In [7]:
areas_verdes_recort = gpd.overlay(
    gdf_areas_verdes,
    gdf_distritos,
    'intersection',
    keep_geom_type=True
)

In [8]:
areas_verdes_recort.sample(2)

,area_avp,id_avp,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry
3982,1274.536990,3841,3550308,São Paulo,355030835,ITAIM BIBI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,10.030646,101452,60889,"POLYGON ((328025.875 7386375.164, 328030.485 7..."
2572,795.736077,2480,3550308,São Paulo,355030863,PIRITUBA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,17.052357,179724,78342,"POLYGON ((325568.428 7402276.592, 325571.054 7..."


# Área dos Polígonos recortados

In [9]:
areas_verdes_recort['area_recort_avp'] = areas_verdes_recort['geometry'].area

# Calcular área por distrito

In [10]:
for distrito in gdf_distritos['CD_DIST']:
    area_avp_dist = sum(
        areas_verdes_recort['area_recort_avp']
        .loc[areas_verdes_recort['CD_DIST']==distrito]
    )

    gdf_distritos.loc[gdf_distritos['CD_DIST']==distrito,'area_avp_dist']= area_avp_dist
    

In [11]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_avp_dist
78,3550308,São Paulo,355030879,SOCORRO,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.261579,38051,16549,"POLYGON ((326832.519 7378799.896, 326814.871 7...",856760.760121
81,3550308,São Paulo,355030882,TUCURUVI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.448038,99559,46695,"POLYGON ((337127.372 7403854.043, 337182.947 7...",149589.056662


In [12]:
gdf_distritos.loc[gdf_distritos['CD_DIST']=='355030812']

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_avp_dist
11,3550308,São Paulo,355030812,BUTANTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.887306,51715,25984,"POLYGON ((325430.01 7391267.565, 325425.704 73...",809343.01376


In [13]:
confere=areas_verdes_recort.loc[areas_verdes_recort['CD_DIST']=='355030812']
confere.sample(2)

,area_avp,id_avp,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_recort_avp
2086,795.635139,2008,3550308,São Paulo,355030812,BUTANTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.887306,51715,25984,"POLYGON ((324299.999 7393820.703, 324301.869 7...",795.635139
1722,38.470859,1657,3550308,São Paulo,355030812,BUTANTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.887306,51715,25984,"POLYGON ((323705.334 7391907.495, 323704.806 7...",38.470859


In [14]:
confere.shape

(332, 17)

In [15]:
sum(confere['area_recort_avp'])

809343.0137602623

# Salvar gdf

In [ ]:
save_parquet_excel(
    gdf_distritos,
    fname='distritos_areas_verdes_publicas',
    data_path = assets_path
    
)